In [1]:
!wandb login a9ef7ce88795773489b968fbc57e3f1d17a8532f

wandb: Appending key for api.wandb.ai to your netrc file: /home/congyu/.netrc
Successfully logged in to Weights & Biases!


In [2]:
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

from keras.models import load_model

Using TensorFlow backend.
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/congyu/.virtua

In [3]:
# Import wandb libraries
import wandb
from wandb.keras import WandbCallback

In [4]:
# data

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# data pre-processing
X_train = X_train.reshape(X_train.shape[0], -1) / 255.   # normalize
X_test = X_test.reshape(X_test.shape[0], -1) / 255.      # normalize
print(y_train[:5])
y_train = to_categorical(y_train, num_classes=10)
print(y_train[:5])
y_test = to_categorical(y_test, num_classes=10)

[5 0 4 1 9]
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [10]:
hyperparameter_defaults = dict(
    n_hidden_units=32,
    epochs=12,
)

# Initialize wandb
wandb.init(project="basicNN", config=hyperparameter_defaults, name="test")
config = wandb.config

In [11]:
# model

model = Sequential()
model.add(Dense(config.n_hidden_units, input_dim=784, activation="relu"))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer="rmsprop", loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [12]:
# training

model.fit(X_train, y_train, epochs=config.epochs, batch_size=32, callbacks=[WandbCallback()])

Epoch 1/12
 2112/60000 [>.............................] - ETA: 26s - loss: 1.3784 - accuracy: 0.6378

/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.679228). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)
/home/congyu/.virtualenvs/congyuml/lib/python3.6/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.341525). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


60000/60000 [==============================] - 4s 63us/step - loss: 0.3423 - accuracy: 0.9051
Epoch 2/12
60000/60000 [==============================] - 3s 47us/step - loss: 0.1887 - accuracy: 0.9456
Epoch 3/12
60000/60000 [==============================] - 3s 48us/step - loss: 0.1535 - accuracy: 0.9558
Epoch 4/12
60000/60000 [==============================] - 3s 48us/step - loss: 0.1340 - accuracy: 0.9617
Epoch 5/12
60000/60000 [==============================] - 3s 51us/step - loss: 0.1218 - accuracy: 0.9661
Epoch 6/12
60000/60000 [==============================] - 3s 49us/step - loss: 0.1112 - accuracy: 0.9686
Epoch 7/12
60000/60000 [==============================] - 4s 59us/step - loss: 0.1049 - accuracy: 0.9699
Epoch 8/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.0990 - accuracy: 0.9726
Epoch 9/12
60000/60000 [==============================] - 3s 50us/step - loss: 0.0940 - accuracy: 0.9742
Epoch 10/12
60000/60000 [==============================] - 3s 58us

In [13]:
# testing

loss, accuracy = model.evaluate(X_test, y_test)

print('test loss: ', loss)
print('test accuracy: ', accuracy)

10000/10000 [==============================] - 0s 30us/step
test loss:  0.12349827622086741
test accuracy:  0.9681000113487244


In [14]:
# save and load

model.save("baseline.h5")
model = load_model('baseline.h5')